In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
import copy

import numpy as np

sys.path.append('./../..')
from PointCloudUtils import op3, load_ply_as_pc, visualize_pc, measure_pc_centroid, adjust_pc_coords, radian2degree, m2mm

In [3]:
import math
# Checks if a matrix is a valid rotation matrix.
def is_rotation_matrix(R: np.array) -> bool:
    """
    Check???
    :param R: a matrix of 4x4
    :return: a boolean, ???
    """
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype=R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6

def rotation_matrix_to_euler_angles(R):
    """
    Measure rotations around x, y and z from transformation matrix
    :param R: a rotation matrix
    :return: an array of 3 values that describe rotations around x, y and z axis, unit is "radian"
    """
    assert (is_rotation_matrix(R))

    sy = math.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])

    singular = sy < 1e-6

    if not singular:
        x = math.atan2(R[2, 1], R[2, 2])
        y = math.atan2(-R[2, 0], sy)
        z = math.atan2(R[1, 0], R[0, 0])
    else:
        x = math.atan2(-R[1, 2], R[1, 1])
        y = math.atan2(-R[2, 0], sy)
        z = 0
    return np.array([x, y, z])

In [4]:
def execute_global_registration(source_down: op3.PointCloud, target_down: op3.PointCloud, 
                                source_fpfh: op3.PointCloud, target_fpfh: op3.PointCloud, 
                                voxel_size: float, verbose=False):
    """
    Do global matching, find gross transformation form source to target
    :param source_down, target_down: 2 objects of Open3D, that are point clouds of source and target after down-sampling
    :param source_fpfh, target_fpfh: 2 objects of Open3D, that are point cloud features of source and target
    :param voxel_size: a float value, that is how sparse you want the sample points is
    :param verbose: a boolean value, display notification and visualization when True and no notification when False
    :return: a transformation object
    """
    distance_threshold = voxel_size * 1.5
    if verbose:
        print(":: RANSAC registration on downsampled point clouds.")
        print("   Since the downsampling voxel size is %.3f," % voxel_size)
        print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = op3.registration_ransac_based_on_feature_matching(
            source_down, target_down, source_fpfh, target_fpfh,
            distance_threshold,
            op3.TransformationEstimationPointToPoint(False), 4,
            [op3.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            op3.CorrespondenceCheckerBasedOnDistance(distance_threshold)],
            op3.RANSACConvergenceCriteria(4000000, 500))
    return result

In [5]:
def execute_fast_global_registration(source_down: op3.PointCloud, target_down: op3.PointCloud, 
                                     source_fpfh: op3.PointCloud, target_fpfh: op3.PointCloud, 
                                     voxel_size: float, verbose=False):
    """
    Find registertration to transform source point cloud to target point cloud
    :param source, target: 2 objects of Open3D, that are point clouds of source and target
    :param voxel_size: a float value, that is how sparse you want the sample points is
    :param verbose: a boolean value, display notification and visualization when True and no notification when False
    :return: a transformation object
    """
    distance_threshold = voxel_size * 0.5
    if verbose: 
        print(":: Apply fast global registration with distance threshold %.3f" % distance_threshold)
    result = op3.registration_fast_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, 
        op3.FastGlobalRegistrationOption(maximum_correspondence_distance = distance_threshold))
    return result

In [13]:
from PointCloudUtils import sample_point_cloud_feature, refine_registration, visualize_registration
def global_icp(source: op3.PointCloud, target: op3.PointCloud, voxel_size = 0.005, verbose=False):
    """
    Find registertration to transform source point cloud to target point cloud
    :param source, target: 2 objects of Open3D, that are point clouds of source and target
    :param voxel_size: a float value, that is how sparse you want the sample points is
    :param verbose: a boolean value, display notification and visualization when True and no notification when False
    :return: a transformation object
    """
    if verbose: visualize_registration(source=source, target=target, transformation=np.identity(4))  # visualize point cloud
    
    # downsample data
    source_down, source_fpfh = sample_point_cloud_feature(point_cloud=source, voxel_size=voxel_size, verbose=verbose)
    target_down, target_fpfh = sample_point_cloud_feature(point_cloud=target, voxel_size=voxel_size, verbose=verbose)

    # 1st: gross matching(RANSAC)
    result_ransac = execute_global_registration(source_down=source_down, target_down=target_down, 
                                                source_fpfh=source_fpfh, target_fpfh=target_fpfh, 
                                                voxel_size=voxel_size, verbose=verbose)
    if verbose: visualize_registration(source=source_down, target=target_down, transformation=result_ransac.transformation)

    # 2nd: fine-tune matching(ICP)
    result_icp = refine_registration(source=source_down, target=target_down, voxel_size=voxel_size, gross_matching=result_ransac)
    if verbose: visualize_registration(source=source_down, target=target_down, transformation=result_icp.transformation)
    return result_icp

In [14]:
def fast_global_icp(source: op3.PointCloud, target: op3.PointCloud, voxel_size = 0.005, verbose=False):
    """
    Find registertration to transform source point cloud to target point cloud
    :param source, target: 2 objects of Open3D, that are point clouds of source and target
    :param voxel_size: a float value, that is how sparse you want the sample points is
    :param verbose: a boolean value, display notification and visualization when True and no notification when False
    :return: a transformation object
    """
    if verbose: visualize_registration(source=source, target=target, transformation=np.identity(4))  # visualize point cloud
    
    # downsample data
    source_down, source_fpfh = sample_point_cloud_feature(point_cloud=source, voxel_size=voxel_size, verbose=verbose)
    target_down, target_fpfh = sample_point_cloud_feature(point_cloud=target, voxel_size=voxel_size, verbose=verbose)

    # 1st: gross matching(RANSAC)
    result_ransac = execute_fast_global_registration(source_down=source_down, target_down=target_down, 
                                                source_fpfh=source_fpfh, target_fpfh=target_fpfh, 
                                                voxel_size=voxel_size, verbose=verbose)
    if verbose: visualize_registration(source=source_down, target=target_down, transformation=result_ransac.transformation)

    # 2nd: fine-tune matching(ICP)
    result_icp = refine_registration(source=source_down, target=target_down, voxel_size=voxel_size, gross_matching=result_ransac)
    if verbose: visualize_registration(source=source_down, target=target_down, transformation=result_icp.transformation)
    return result_icp

# Illustrate how to use surface matching

In [8]:
path_point_cloud1 = './../../data/72_scenes_of_pipe/pipe/1001.ply'
path_point_cloud2 = './../../data/72_scenes_of_pipe/pipe/1002.ply'
path_point_cloud3 = './../../data/72_scenes_of_pipe/pipe/1003.ply'

In [9]:
# Load data
point_cloud1 = load_ply_as_pc(file_path=path_point_cloud1)
point_cloud2 = load_ply_as_pc(file_path=path_point_cloud2)
point_cloud3 = load_ply_as_pc(file_path=path_point_cloud3)
# point_cloud.visualize()

In [10]:
visualize_pc(point_cloud1, point_cloud2, point_cloud3)

### Should adjust z axis to have better matching result

In [11]:
point_cloud1 = adjust_pc_coords(point_cloud=point_cloud1, coord=[0, 0, 0.850])
point_cloud2 = adjust_pc_coords(point_cloud=point_cloud2, coord=[0, 0, 0.850])

### Should adjust model to centroid to have better matching result

In [12]:
centroid1 = measure_pc_centroid(point_cloud=point_cloud1); print(centroid1)
point_cloud1 = adjust_pc_coords(point_cloud=point_cloud1, coord=centroid1[:3])

[-0.07636259 -0.07773111  0.01330988  0.83884553  0.86357271  0.89967349]


### Find matching transformation

In [20]:
xtrans = global_icp(source=point_cloud1, target=point_cloud2, voxel_size=0.002, verbose=True)
print(xtrans)
print('Theta x, Theta y, Theta z(in Degree): \n {}'.format(radian2degree(rotation_matrix_to_euler_angles(xtrans.transformation[:3, :3]))))
print('Translation(in miliMeters): \n {}'.format(m2mm(xtrans.transformation[:3, 3]) - m2mm(centroid1[:3])))

:: Downsample with a voxel size 0.002.
:: Estimate normal with search radius 0.004.
:: Compute FPFH feature with search radius 0.010.
:: Downsample with a voxel size 0.002.
:: Estimate normal with search radius 0.004.
:: Compute FPFH feature with search radius 0.010.
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.002,
   we use a liberal distance threshold 0.003.
registration::RegistrationResult with fitness = 0.120092, inlier_rmse = 0.000578, and correspondence_set size of 104
Access transformation to get result.
Theta x, Theta y, Theta z(in Degree): 
 [-0.88849678  3.00138422 -7.37927514]
Translation(in miliMeters): 
 [48.73127258 -3.35798609  0.75714762]


In [14]:
xtrans = fast_global_icp(source=point_cloud1, target=point_cloud2, voxel_size=0.005, verbose=True)
print(xtrans)
print('Theta x, Theta y, Theta z(in Degree): \n {}'.format(radian2degree(rotation_matrix_to_euler_angles(xtrans.transformation[:3, :3]))))
print('Translation(in miliMeters): \n {}'.format(m2mm(xtrans.transformation[:3, 3]) - m2mm(centroid1[:3])))

:: Downsample with a voxel size 0.005.
:: Estimate normal with search radius 0.010.
:: Compute FPFH feature with search radius 0.025.
:: Downsample with a voxel size 0.005.
:: Estimate normal with search radius 0.010.
:: Compute FPFH feature with search radius 0.025.
:: Apply fast global registration with distance threshold 0.003
RegistrationResult with fitness = 0.318584, inlier_rmse = 0.001486, and correspondence_set size of 72
Access transformation to get result.
Theta x, Theta y, Theta z(in Degree): 
 [0.70988453 3.58042679 3.73361342]
Translation(in miliMeters): 
 [47.55943785 -0.35614765  1.74757792]


In [40]:
%timeit xtrans = global_icp(source=point_cloud1, target=point_cloud2, voxel_size=0.005, verbose=False)

248 ms ± 36.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
%timeit xtrans = fast_global_icp(source=point_cloud1, target=point_cloud2, voxel_size=0.005, verbose=False)

19.9 ms ± 1.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
